In [ ]:
#create dataset
%%bigquery
CREATE SCHEMA IF NOT EXISTS fraud_data_raw
OPTIONS(
 location="us");

#create Table
CREATE OR REPLACE EXTERNAL TABLE `qwiklabs-gcp-02-e82d432d54e9.fraud_data_raw.fraud_data`
OPTIONS (
  format = 'CSV',
  uris = ['gs://labs.roitraining.com/data-to-ai-workshop/fraud_data_raw.csv'],
  skip_leading_rows = 1,
  allow_jagged_rows = false,
  allow_quoted_newlines = false,
  field_delimiter = ',',
  max_bad_records = 10
)


Query is running:   0%|          |

""


In [42]:
#one-hot encode
%%bigquery
CREATE or REPLACE TABLE `qwiklabs-gcp-02-e82d432d54e9`.fraud_data_raw.fraud_training_data AS
SELECT
*   EXCEPT(
    Previous_Assistance_Received, Supporting_Doc_Verified
    ),
  CASE
    WHEN Employment_Status = 'Employed' THEN 1
    ELSE 0
  END AS Employment_Status_Employed,
  CASE
    WHEN Employment_Status = 'Unemployed' THEN 1
    ELSE 0
  END AS Employment_Status_Unemployed,
  CASE
    WHEN Employment_Status = 'Self-Employed' THEN 1
    ELSE 0
  END AS Employment_Status_SelfEmployed,
  CASE
    WHEN Device_Type = 'Mobile' THEN 1
    ELSE 0
  END AS Device_Type_Mobile,
  CASE
    WHEN Device_Type = 'Desktop' THEN 1
    ELSE 0
  END AS Device_Type_Desktop,
  CASE
    WHEN Device_Type = 'Tablet' THEN 1
    ELSE 0
  END AS Device_Type_Tablet,
  #put ages into bins
  CASE
    WHEN Age <= 18 THEN '0-18 (Child/Teen)'
    WHEN Age BETWEEN 19 AND 25 THEN '19-25 (Young Adult)'
    WHEN Age BETWEEN 26 AND 40 THEN '26-40 (Adult)'
    WHEN Age BETWEEN 41 AND 60 THEN '41-60 (Middle-aged)'
    WHEN Age > 60 THEN '61+ (Senior)'
    ELSE 'Unknown'
  END AS Age_Bin,
  #Create a calculated field called Time_Since_Previous_Assistance
  DATE_DIFF(CURRENT_DATE(), Previous_Assistance_Date, DAY) AS Time_Since_Previous_Assistance,
  #Create a field called Income-to-Amount-Requested which is the ratio of those fields
  CASE WHEN Amount_Requested = 0 THEN NULL ELSE CAST(Income AS FLOAT64) / Amount_Requested END AS `Income-to-Amount-Requested`,
  #Change True/False fields to 0s and 1
  CASE WHEN Previous_Assistance_Received = true then 1 ELSE 0 END AS Previous_Assistance_Received_bool,
  CASE WHEN Supporting_Doc_Verified = true then 1 ELSE 0 END AS Supporting_Doc_Verified_bool,


FROM
  `qwiklabs-gcp-02-e82d432d54e9.fraud_data_raw.fraud_data`;

Query is running:   0%|          |

""


In [ ]:


  #Change True/False fields to 0s and 1
  CASE WHEN Previous_Assistance_Received = 'true' then 1 ELSE 0 END AS Previous_Assistance_Received_bool,
  CASE WHEN Supporting_Doc_Verified = 'true' then 1 ELSE 0 END AS Supporting_Doc_Verified_bool,

Query is running:   0%|          |

""
